In [1]:
!pip3 install datasets
!clear

     |################################| 298 kB 20.6 MB/s eta 0:00:01
     |################################| 106 kB 29.0 MB/s eta 0:00:01
     |################################| 9.5 MB 30.3 MB/s eta 0:00:01
     |################################| 1.1 MB 30.4 MB/s eta 0:00:01
     |################################| 76 kB 9.7 MB/s  eta 0:00:01
     |################################| 25.6 MB 7.9 MB/s eta 0:00:011
     |################################| 243 kB 31.1 MB/s eta 0:00:01
     |################################| 86 kB 11.4 MB/s eta 0:00:01
     |################################| 132 kB 29.2 MB/s eta 0:00:01
     |################################| 61 kB 1.2 MB/s  eta 0:00:01
     |################################| 503 kB 27.6 MB/s eta 0:00:01
     |################################| 270 kB 29.2 MB/s eta 0:00:01
     |################################| 191 kB 32.1 MB/s eta 0:00:01
     |################################| 159 kB 30.5 MB/s eta 0:00:01
     |##############################

In [2]:
from utils import get_geo880_classification
dataset = get_geo880_classification()

In [3]:
from transformers import AutoTokenizer, BertForSequenceClassification

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
dataset_dict = dataset.train_test_split(train_size=train_size, test_size=test_size)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 1584
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 176
    })
})

In [5]:
print('{:>5,} training samples'.format(train_size))
print('{:>5,} test samples'.format(test_size))

1,584 training samples
  176 test samples


In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
import torch
torch.cuda.empty_cache()

In [9]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=8,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    logging_steps=200,
    evaluation_strategy="steps",
    output_dir="./checkpoints/bert/"
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test'])
trainer.train()

Step,Training Loss,Validation Loss
200,0.000004,0.000000
400,0.000000,0.000000
600,0.000000,0.000000
800,0.000000,0.000000
1000,0.000000,0.000000
1200,0.000000,0.000000
1400,0.000000,0.000000
1600,0.000000,0.000000
1800,0.000000,0.000000


TrainOutput(global_step=1980, training_loss=4.4402282454562607e-07)

In [27]:
results = trainer.predict(tokenized_datasets['test'])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [38]:
tokenizer.decode(tokenized_datasets['test']["input_ids"][25])

'[CLS] A, ( state ( A ), next _ to ( A, B ), state ( B ), next _ to ( B, C ), state ( C ), next _ to ( C, D ), const ( D, stateid ( florida ) ) ) [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [37]:
results.label_ids[25]

0